In [7]:
import json
import os
from pprint import pprint

import pyomo.environ as pe
import pyomo.opt as po
import yaml

In [8]:
cfl_blk_demand_filename = os.path.join('data', 'cfl_blk_by_demand.json')

# Setup Solver

In [9]:
solver = po.SolverFactory('glpk')

# Capacitated Facility Location
To demonstrate blocks, let's setup and solve an instance of the CFL model.

Sets:
- $I$: supply sites, indexed by $i$
- $J$: demand sites, indexed by $j$

Parameters:
- $s_i$: supply capacity of supply site $i$
- $d_j$: demand required by demand site $j$
- $f_i$: fixed cost to open supply site $i$
- $c_{ij}$: variable cost to transport from supply site $i$ to demand site $j$

Variables:
- $x_{ij}$ - quantity of product to ship from supply site $i$ to demand site $j$
- $y_i$ - 0/1 decision variable indicating that supply site $i$ is producing

Model:
$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\
\text{subject to  }
& \sum_{i \in I} x_{ij} \ge d_j && \forall j \in J \\
& \sum_{j \in J} x_{ij} \le s_i y_i && \forall i \in I \\
& x \in \mathbb{R}_+^{|I| \times |J|} && \\
& y \in \{0, 1\}^{|I|} && \\
\end{alignat*}
$$

# Implementation with Blocks Indexed by Demand Sites

In [10]:
with open(cfl_blk_demand_filename) as fh:
    data = json.load(fh)
pprint(data)

{'demand_data': {'a': {'demand': 80, 'variable_cost': {'1': 4, '2': 6, '3': 9}},
                 'b': {'demand': 270,
                       'variable_cost': {'1': 5, '2': 4, '3': 7}},
                 'c': {'demand': 250,
                       'variable_cost': {'1': 6, '2': 3, '3': 4}},
                 'd': {'demand': 160,
                       'variable_cost': {'1': 8, '2': 5, '3': 3}},
                 'e': {'demand': 180,
                       'variable_cost': {'1': 10, '2': 8, '3': 4}}},
 'demand_sites': ['a', 'b', 'c', 'd', 'e'],
 'fixed_cost': {'1': 1000, '2': 1000, '3': 1000},
 'supply': {'1': 500, '2': 500, '3': 500},
 'supply_sites': ['1', '2', '3']}


In [11]:
model = pe.ConcreteModel("Blocks (by Demand Site)")

# sets
model.I = pe.Set(initialize=data['supply_sites'])
model.J = pe.Set(initialize=data['demand_sites'])

# parameters (not indexed in J)
model.s = pe.Param(model.I, initialize=data['supply'])
model.f = pe.Param(model.I, initialize=data['fixed_cost'])

# variables (not indexed in J)
model.y = pe.Var(model.I, domain=pe.Binary)

# blocks (indexed in J)
def blk_demand(block, j):
    blk_data = data['demand_data'][j]
    I = block.model().I # borrow the set of supply sites from the overarching model
    block.d = pe.Param(initialize=blk_data['demand'])
    block.c = pe.Param(I, initialize=blk_data['variable_cost'])
    block.x = pe.Var(I, domain=pe.NonNegativeReals)
    block.con_satisfaction = pe.Constraint(expr=(sum(block.x[i] for i in I) >= block.d))
    block.variable_cost = sum(block.c[i] * block.x[i] for i in I)
model.blk_demand = pe.Block(model.J, rule=blk_demand)

# constraints (not indexed in J)
def con_transportation(model, i):
    return sum(model.blk_demand[j].x[i] for j in model.J) <= model.s[i] * model.y[i]
model.con_transportation = pe.Constraint(model.I, rule=con_transportation)

# objective
def obj_min_cost(model):
    return sum(model.f[i] * model.y[i] for i in model.I)\
        + sum(model.blk_demand[j].variable_cost for j in model.J)
model.obj_min_cost = pe.Objective(sense=pe.minimize, rule=obj_min_cost)

In [12]:
result = solver.solve(model)
model.display()

Model 'Blocks (by Demand Site)'

  Variables:
    y : Size=3, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obj_min_cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 5610.0

  Constraints:
    con_transportation : Size=3
        Key : Lower : Body  : Upper
          1 :  None :   0.0 :   0.0
          2 :  None :   0.0 :   0.0
          3 :  None : -60.0 :   0.0

  Blocks:
    Block blk_demand[a]
    
      Variables:
        x : Size=3, Index=I
            Key : Lower : Value : Upper : Fixed : Stale : Domain
              1 :     0 :   0.0 :  None : False : False : NonNegativeReals
              2 :     0 :  80.0 :  None : False : False : NonNegativeReals
              3 :     0 :   0.0 :  None : False : False : NonNegati